In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import datetime as dt
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import cpi
import warnings
warnings.filterwarnings("ignore")


# Initialize Data

In [ ]:
raw_data = pd.read_csv('MedianSalePrice.csv')
raw_data = raw_data[(raw_data['StateName'] == 'IL') & (raw_data['RegionName'] == 'Chicago, IL')]
raw_data = raw_data.iloc[:, 6:]
raw_data = np.transpose(raw_data)
raw_data.columns = ['Raw Median Sale Price']
raw_data.index = pd.to_datetime(raw_data.index)
raw_data = raw_data[256:725]
raw_data.plot(figsize=(20, 5))
plt.grid()
plt.legend(loc='best')
plt.title('Raw Data')
plt.show(block=False)
raw_data['raw_date'] = raw_data.index
raw_data[["Year", "Month", "Day"]] = raw_data['raw_date'].astype("string").str.split("-", expand=True)
raw_data['Year'] = raw_data['Year'].astype(int)


def inflate_column(data, column):
    """
    Adjust for inflation the series of values in column of the   
    dataframe data
    """
    return data.apply(lambda x: cpi.inflate(x[column], x['Year'], to=2021), axis=1)


raw_data['Inflation-Adjusted Median Sale Price'] = inflate_column(raw_data, 'Raw Median Sale Price')
raw_data[['Raw Median Sale Price','Inflation-Adjusted Median Sale Price']].plot(figsize=(20, 5))
plt.grid()
plt.legend(loc='best')
plt.title('Inflation-Adjusted Data vs Raw Data')
plt.show(block=False)
train_data = raw_data[:417]
train_data = train_data[['Inflation-Adjusted Median Sale Price']]
test_data = raw_data[417-52:]
test_data = test_data[['Inflation-Adjusted Median Sale Price']]

In [ ]:
train_data

In [ ]:
test_data

# Raw Data

In [ ]:
# train_data.plot(figsize=(20, 5))
# plt.grid()
# plt.legend(loc='best')
# plt.title('Inflation Adjusted - Training Data')
# plt.show(block=False)

In [ ]:
train_data

# Fit Linear Model and get residuals

In [ ]:
train_data['Date'] = train_data.index
train_data['Date'] = train_data['Date'].map(dt.datetime.toordinal)
# Fit Polynomial Regression
reg = np.poly1d(np.polyfit(train_data['Date'], train_data['Inflation-Adjusted Median Sale Price'], 3))
print(reg)
train_data['Polynomial Regression Fit'] = reg(train_data['Date'])
train_data[['Inflation-Adjusted Median Sale Price', 'Polynomial Regression Fit']].plot(figsize=(20, 5))
plt.grid()
plt.legend(loc='best')
plt.title('Inflation-Adjusted Median Sale Price vs. Polynomial Model')
plt.show(block=False)

# Residuals

In [ ]:
train_data['Residuals'] = train_data['Inflation-Adjusted Median Sale Price'] - train_data['Polynomial Regression Fit']
train_data[['Residuals']].plot(figsize=(20, 5))
plt.grid()
plt.legend(loc='best')
plt.title('Residuals')
plt.show(block=False)

# Remove 52 week cycle based on spectrum

In [ ]:
train_data['Diff'] = train_data['Residuals'].diff(52)
removed_train_data = train_data[:52]
train_data = train_data[52:]
train_data[['Diff']].plot(figsize=(20, 5))
plt.grid()
plt.legend(loc='best')
plt.title('Yearly Cycle and Polynomial Trend Removed')
plt.show(block=False)


In [ ]:
# train_data['Diff'] = train_data['Residuals'].diff(1)
# removed_train_data = train_data[:1]
# train_data = train_data[1:]
# train_data[['Diff']].plot(figsize=(20, 5))
# plt.grid()
# plt.legend(loc='best')
# plt.title('Diff')
# plt.show(block=False)

# train_data['Diff'] = train_data['Residuals']


# Stationary

In [ ]:
adfuller(train_data['Diff'])

# Analyze Residuals

In [ ]:
rcParams['figure.figsize'] = 20, 10
decomposition = sm.tsa.seasonal_decompose(train_data['Diff'], model='additive')  # additive seasonal index
fig = decomposition.plot()
plt.show()


# Plot PACF and ACF

In [ ]:
plt.figure(figsize=(20, 5))
plt.grid()
plot_acf(train_data['Diff'], ax=plt.gca(), lags=15, zero=False)
plt.show()
plt.figure(figsize=(20, 5))
plt.grid()
plot_pacf(train_data['Diff'], ax=plt.gca(), lags=15, method='ywm', zero=False)
plt.show()


# Train ARIMA Model

In [ ]:
train_data_arima = train_data[['Diff']]

In [ ]:
# Train ARIMA Model
scoring = []
for p in np.arange(0,10,1):
        for q in np.arange(0, 10, 1):
            try:
                model = SARIMAX(train_data_arima, order=(p, 0, q)).fit()
                scoring.append((p,q,model.aic, model.bic))
            except:
                pass
scoring.sort(key = lambda x: x[2])
print(scoring[:10])
# # Predict Data
model = SARIMAX(train_data_arima, order=(4, 0, 3)).fit()
plot_data = train_data_arima.copy()
plot_data['Residuals_Predictions'] = model.predict().tolist()
plot_data = plot_data[1:]
# Plot Prediction
plot_data.plot(figsize=(20, 5))
plt.grid()
plt.legend(loc='best')
plt.title('Comparison between actual and predicted residuals')
plt.show(block=False)


In [ ]:
print(model.polynomial_ar)
print(model.polynomial_ma)

In [ ]:
print("MAE: ", mean_absolute_error(plot_data['Diff'], plot_data['Residuals_Predictions']))
print("RMSE: ", np.sqrt(mean_squared_error(plot_data['Diff'], plot_data['Residuals_Predictions'])))


# Test Set

In [ ]:
train_data.mean()

In [ ]:
# # train_data.mean()  # 243959.484307
# test_data['Mean'] = [train_data.mean()[0]] * len(test_data)
# test_data['Polynomial Regression Prediction'] = reg(test_data.index.map(dt.datetime.toordinal))
# test_data['Residuals'] = test_data['Inflation-Adjusted Median Sale Price'] - test_data['Polynomial Regression Prediction']
# test_data['Diff'] = test_data['Residuals'].diff(52)

# training_samples = len(train_data)
# testing_samples = len(test_data[52:])
# frames = [train_data[['Diff']], test_data[['Diff']][52:]]
# total_frame = pd.concat(frames)
# predictions = [float("NaN")] * 52
# for i in range(0, testing_samples):
#     model = SARIMAX(total_frame[:training_samples+i],order=(9, 0, 9)).fit()
#     predictions.append(model.forecast()[0])
# test_data['Diff_Predictions'] = predictions
# x, x_diff = test_data['Residuals'].iloc[0:52], test_data['Diff_Predictions'].iloc[52:]
# test_data['Residuals_Predictions'] = np.r_[x, x_diff].cumsum().astype(int)
# test_data['Overall_Predictions'] = test_data['Residuals_Predictions'] + test_data['Polynomial Regression Prediction']
# og_data = test_data

# train_data.mean()  # 243959.484307
test_data['Mean'] = [train_data.mean()[0]] * len(test_data)
test_data['Polynomial Regression Prediction'] = reg(
    test_data.index.map(dt.datetime.toordinal))
test_data['Residuals'] = test_data['Inflation-Adjusted Median Sale Price'] - \
    test_data['Polynomial Regression Prediction']
test_data['Diff'] = test_data['Residuals'].diff(52)

training_samples = len(train_data)
testing_samples = len(test_data[52:])
frames = [train_data[['Diff']], test_data[['Diff']][52:]]
total_frame = pd.concat(frames)
predictions = [float("NaN")] * 52
for i in range(0, testing_samples):
    model = SARIMAX(total_frame[:training_samples+i], order=(9, 0, 9)).fit()
    test_data['Diff_Predictions'] = (model.forecast(len(test_data[52:])))
    break
# test_data['Diff_Predictions'] = predictions
x, x_diff = test_data['Residuals'].iloc[0:52], test_data['Diff_Predictions'].iloc[52:]
test_data['Residuals_Predictions'] = np.r_[x, x_diff].cumsum().astype(int)
test_data['Overall_Predictions'] = test_data['Residuals_Predictions'] + \
    test_data['Polynomial Regression Prediction']
og_data = test_data


In [ ]:
og_data = test_data[52:52+4]
og_data[['Inflation-Adjusted Median Sale Price', 'Overall_Predictions']].plot()

In [ ]:
# og_data = test_data
print("Mean")
print("MAE: ", mean_absolute_error(og_data ['Inflation-Adjusted Median Sale Price'], og_data ['Mean']))
print("RMSE: ", np.sqrt(mean_squared_error(og_data ['Inflation-Adjusted Median Sale Price'],og_data ['Mean'])))
print("Polynomial")
print("MAE: ", mean_absolute_error(og_data ['Inflation-Adjusted Median Sale Price'], og_data ['Polynomial Regression Prediction']))
print("RMSE: ", np.sqrt(mean_squared_error(og_data ['Inflation-Adjusted Median Sale Price'], og_data ['Polynomial Regression Prediction'])))
print("ARIMA")
print("MAE: ", mean_absolute_error(og_data ['Inflation-Adjusted Median Sale Price'], og_data ['Overall_Predictions']))
print("RMSE: ", np.sqrt(mean_squared_error(og_data ['Inflation-Adjusted Median Sale Price'], og_data ['Overall_Predictions'])))


# Add Linear Regression Back In

In [ ]:
prediction_data = plot_data.copy()
prediction_data['Linear_Regression_Predictions'] = reg(np.array(train_data['Date'][1:]).reshape(-1, 1))
prediction_data['Predictions'] = prediction_data['Linear_Regression_Predictions'] + prediction_data['Residuals_Predictions']
prediction_data['Truth_Value'] = train_data['normalized_sale_price'][1:]
# Plot Prediction
prediction_data[['Predictions', 'Truth_Value']].plot(figsize=(20, 5))
plt.grid()
plt.legend(loc='best')
plt.title('Comparison between Predicted Median Sale Price')
plt.show(block=False)
print("MAE: ", mean_absolute_error(prediction_data['Truth_Value'], prediction_data['Predictions']))
print("RMSE: ", np.sqrt(mean_squared_error(prediction_data['Truth_Value'], prediction_data['Predictions'])))
